In [1]:
import dask
import dask_image.imread
import dask.distributed
import skimage.io as io

import numpy as np

from sklearn.svm import SVC
from sklearn import metrics

import os
import sys
current = os.path.dirname(os.path.realpath(os.getcwd()))
parent = os.path.dirname(current)
sys.path.append(parent)
from Emotions_Detection.Models.features import lpq, lpq_plus, PHOG_Algorithm, LPQ, LPQPlus
from Emotions_Detection.Assets.CommonFuntions import read_data

In [2]:
# Constants and Vaiables
IMG_TRAIN_PATH = "../Assets/data/train"
IMG_TEST_PATH = "../Assets/data/test"
HAPPY_PATH = "../Assets/data/train/happy/"

train_folders = os.listdir(IMG_TRAIN_PATH)  # Emotions Folders
test_folders = os.listdir(IMG_TEST_PATH)  # Emotions Folders

In [3]:
train_imgs, y_train = read_data(IMG_TRAIN_PATH)

In [4]:
test_imgs, y_test = read_data(IMG_TEST_PATH)

In [5]:
lpq_cls = LPQ()
lpq_feats_train = []
lpq_feats_test = []
for img in train_imgs:
    lpq_feats_train.append(lpq_cls.compute(img))

for img in test_imgs:
    lpq_feats_test.append(lpq_cls.compute(img))

In [6]:
phog_feats_train = []
phog_feats_test = []
for img in train_imgs:
    phog_feats_train.append(PHOG_Algorithm(img))

for img in test_imgs:
    phog_feats_test.append(PHOG_Algorithm(img))

In [7]:
x_train = np.concatenate((lpq_feats_train, phog_feats_train), axis=1)
x_test = np.concatenate((lpq_feats_test, phog_feats_test), axis=1)
x_train.shape, x_test.shape

((28709, 935), (7178, 935))

In [9]:
import pickle
with open('features_files/train_lpq.obj', 'wb') as f:
    pickle.dump(lpq_feats_train, f)
with open('features_files/train_phog.obj', 'wb') as f:
    pickle.dump(phog_feats_train, f)
with open('features_files/test_lpq.obj', 'wb') as f:
    pickle.dump(lpq_feats_test, f)
with open('features_files/test_phog.obj', 'wb') as f:
    pickle.dump(phog_feats_test, f)
with open('features_files/train_lpq_phog.obj', 'wb') as f:
    pickle.dump(x_train, f)
with open('features_files/test_lpq_phog.obj', 'wb') as f:
    pickle.dump(x_test, f)

In [10]:
#Create a svm Classifier
# SVC(C=1, gamma=1, kernel='poly')
clf = SVC(C=1, gamma=1)
print("classifier initialized...")

#Train the model using the training sets
clf.fit(x_train, y_train)
print("classifier Trained...")

#Predict the response for test dataset
y_pred = clf.predict(x_test)

classifier initialized...
classifier Trained...


In [12]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average=None))

Accuracy: 0.3152688771245472
Precision: [0.95454545 1.         0.90816327 0.26582658 0.93478261 0.92857143
 0.98695652]


In [14]:
y_pred

array(['happy', 'happy', 'happy', ..., 'happy', 'surprised', 'happy'],
      dtype='<U9')

In [20]:
len(y_test)

7178